In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib as mpl
import os
from toolz import concat
from matplotlib.lines import Line2D
from collections import Counter
import math
from tqdm import tqdm
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import random
import scipy
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import pinv_extended  
from statsmodels.stats.anova import anova_lm
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from tabulate import tabulate

In [2]:
format_plots()
#format_pizza_plots()

In [3]:
def mm_norm_col(column):
    return (column - column.min()) / (column.max() - column.min())

## arrange data for ontogeny - male and female seperately

In [4]:
## arrange data
keep_syllables = np.loadtxt('to_keep_syllables_raw.txt', dtype=int)

male_df = pd.read_parquet('ontogeny_males_raw_usage_matrix_v00.parquet').astype(float)
male_df = male_df[keep_syllables]
m_df= male_df.groupby(['age','uuid']).mean()


## arrange data for females
keep_syllables = np.loadtxt('to_keep_syllables_raw.txt', dtype=int)

female_df = pd.read_parquet('ontogeny_females_raw_usage_matrix_v00.parquet').astype(float)
female_df = female_df[keep_syllables]


# for female:
f_df= female_df.groupby(['age','uuid']).mean()
# Update age value to 52 for rows with uuid in the list
uuids = ['f1d5dce5-f5cf-4b03-b07d-d9b8c1f034b0','6fa50ac7-6d49-4ef9-9059-3d90bdd8c5d4',
         'e9e05da8-fc59-40f3-a9c8-f57c492c6141','327e7fa1-2237-43d2-b25f-c1801912df33',
        '242bee8e-0ee7-45e7-8a13-678836a4cddb','552ec70f-0676-4471-8ab0-403e1fcf43e3','d6f254af-d55a-427a-96e0-c452a233cbe2']
f_df.reset_index(inplace=True)

age_mapping = {
    93: 94,
    95: 94,
    97: 98,
    98: 98,
    102: 98,
    103: 105,
    105: 105,
    107: 105
}

f_df['age'] = f_df['age'].map(age_mapping).fillna(f_df['age'])
f_df.loc[f_df['uuid'].isin(uuids), 'age'] = 52
# Set the index back
f_df.set_index(['age', 'uuid'], inplace=True)
f_df=f_df.query('age<127')

In [5]:
def tw_anova(data, dependent_var, factor1, factor2):
    """
    Perform two-way ANOVA.
    
    Parameters:
    data (pd.DataFrame): The input data frame.
    dependent_var (str): The name of the dependent variable column.
    factor1 (str): The name of the first factor column.
    factor2 (str): The name of the second factor column.
    
    Returns:
    pd.DataFrame: The ANOVA results.
    """
    
    # Align the data
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]
    # Fit an OLS model on the ranked data
    #model = ols(f'ranked_response ~ {factor1} * {factor2}', data=align_data).fit()
    model = ols(f'ranked_response ~ {factor1} * C({factor2})', data=align_data).fit()

    # Perform ANOVA on the fitted model
    anova_results = sm.stats.anova_lm(model, typ=3)
    
    # Convert results to a DataFrame
    anova_df = pd.DataFrame(anova_results)
    
    return anova_df

def ow_anova(data, dependent_var, factor1):
    """
    Perform two-way ANOVA.
    
    Parameters:
    data (pd.DataFrame): The input data frame.
    dependent_var (str): The name of the dependent variable column.
    factor1 (str): The name of the first factor column.
    factor2 (str): The name of the second factor column.
    
    Returns:
    pd.DataFrame: The ANOVA results.
    """
    
    # Align the data
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]
    # Fit an OLS model on the ranked data
    model = ols(f'ranked_response ~ {factor1}', data=align_data).fit()
    # Perform ANOVA on the fitted model
    anova_results = sm.stats.anova_lm(model, typ=3)
    
    # Convert results to a DataFrame
    anova_df = pd.DataFrame(anova_results)
    
    return anova_df

## arrange data for male and female together

In [6]:
# use only common ages
f_ages = f_df.index.get_level_values('age').unique().to_numpy()
m_ages = m_df.index.get_level_values('age').unique().to_numpy()
cages = list(np.intersect1d(f_ages,m_ages))

m_data = m_df.loc[m_df.index.get_level_values('age').isin(cages)]
m_data['sex'] = 'm'
m_data.set_index('sex',inplace=True,append=True)

f_data = f_df.loc[f_df.index.get_level_values('age').isin(cages)]
f_data['sex'] = 'f'
f_data.set_index('sex',inplace=True,append=True)
data = pd.concat([m_data,f_data])

## change in syllable over age males - one-way anova

In [7]:
#males
data=m_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
s=[]
p=[]
h=[]
a = 0.05
from scipy import stats
for i in syll:
    print(i)
    anova_results = ow_anova(temp, i, 'age')
    #results[i] = anova_results
    #print(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))
    cp = anova_results['PR(>F)']['age']*n
    if cp<a:
        s.extend([i])
        p.extend([cp])
        h.extend([h])
print('number of syllables changing over time for males:', str(len(s)))

0
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.00230939  |    1 | 206.49   |   3.30291e-37 |
| age       | 0.000189986 |    1 |  16.9873 |   4.69123e-05 |
| Residual  | 0.00395914  |  354 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
2
+-----------+------------+------+----------+---------------+
|           |     sum_sq |   df |        F |        PR(>F) |
|-----------+------------+------+----------+---------------|
| Intercept | 0.11296    |    1 | 433.658  |   1.89464e-63 |
| age       | 0.00641593 |    1 |  24.6309 |   1.08089e-06 |
| Residual  | 0.0922108  |  354 | nan      | nan           |
+-----------+------------+------+----------+---------------+
4
+-----------+------------+------+-----------+---------------+
|           |     sum_sq |   df |         F |        PR(>F) |
|--------

+-----------+------------+------+----------+---------------+
|           |     sum_sq |   df |        F |        PR(>F) |
|-----------+------------+------+----------+---------------|
| Intercept | 0.0172412  |    1 |  61.6394 |   4.96067e-14 |
| age       | 0.00355279 |    1 |  12.7016 |   0.000415422 |
| Residual  | 0.0990178  |  354 | nan      | nan           |
+-----------+------------+------+----------+---------------+
60
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.00360713  |    1 | 471.222  |   4.87978e-67 |
| age       | 0.000231418 |    1 |  30.2317 |   7.3538e-08  |
| Residual  | 0.00270981  |  354 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
61
+-----------+-----------+------+----------+---------------+
|           |    sum_sq |   df |        F |        PR(>F) |
|-----------+

In [8]:
#males
data=m_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

# Initialize variables
results = []
dependent_vars = data.columns
n_tests = len(dependent_vars)  # Number of ANOVA tests

# Perform ANOVA for each dependent variable
for i, k in enumerate(dependent_vars):
    print(k)
    anova_results = ow_anova(temp, k, 'age')
    anova_results['Dependent Variable'] = k  # Add dependent variable to the results
    results.append(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))

# Combine results into a DataFrame
results_df = pd.concat(results).reset_index()
results_df.rename(columns={'index': 'Parameter'}, inplace=True)

# Move the 'Dependent Variable' column to the first position
cols = ['Dependent Variable'] + [col for col in results_df.columns if col != 'Dependent Variable']
results_df = results_df[cols]

# Apply Bonferroni correction to the p-values
results_df['Bonferroni_corrected_p'] = results_df['PR(>F)'] * n_tests
results_df['Bonferroni_corrected_p'] = results_df['Bonferroni_corrected_p'].apply(lambda p: min(p, 1))  # p-values should not exceed 1

# Save DataFrame to CSV file
results_df.to_csv('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/stats/anova_results_syllable_over_time_ontogeny_males.csv', index=False)

print("ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'")

0
+-----------+-------------+------+----------+---------------+----------------------+
|           |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+---------------+----------------------|
| Intercept | 0.00230939  |    1 | 206.49   |   3.30291e-37 |                    0 |
| age       | 0.000189986 |    1 |  16.9873 |   4.69123e-05 |                    0 |
| Residual  | 0.00395914  |  354 | nan      | nan           |                    0 |
+-----------+-------------+------+----------+---------------+----------------------+
2
+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.11296    |    1 | 433.658  |   1.89464e-63 |                    2 |
| age       | 0.00641593 |    1 |  24.6309 |   1.08089e-06 |     

+-----------+-------------+------+----------+---------------+----------------------+
|           |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+---------------+----------------------|
| Intercept | 0.00647859  |    1 | 297.914  |   7.20506e-49 |                   27 |
| age       | 0.000348602 |    1 |  16.0303 |   7.59771e-05 |                   27 |
| Residual  | 0.00769827  |  354 | nan      | nan           |                   27 |
+-----------+-------------+------+----------+---------------+----------------------+
30
+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.00306575  |    1 | 298.009   |   7.02054e-49 |                   30 |
| age       | 9.60776e-05 |    1 |   9.33933 |   0.0024131

+-----------+-----------+------+---------+---------------+----------------------+
|           |    sum_sq |   df |       F |        PR(>F) |   Dependent Variable |
|-----------+-----------+------+---------+---------------+----------------------|
| Intercept | 0.485145  |    1 | 770.763 |   6.98911e-91 |                   62 |
| age       | 0.0760432 |    1 | 120.812 |   2.23932e-24 |                   62 |
| Residual  | 0.22282   |  354 | nan     | nan           |                   62 |
+-----------+-----------+------+---------+---------------+----------------------+
63
+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.00387054  |    1 | 313.664   |   1.03913e-50 |                   63 |
| age       | 5.90465e-05 |    1 |   4.78506 |   0.0293612   |                 

+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.00360342  |    1 | 298.409   |   6.29577e-49 |                   93 |
| age       | 1.46401e-05 |    1 |   1.21239 |   0.271607    |                   93 |
| Residual  | 0.0042747   |  354 | nan       | nan           |                   93 |
+-----------+-------------+------+-----------+---------------+----------------------+
95
+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.0163296   |    1 | 471.661   |   4.44117e-67 |                   95 |
| age       | 3.57556e-05 |    1 |   1.03276 |   0.

In [9]:
#females
data=f_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
s=[]
p=[]
h=[]
a = 0.05
from scipy import stats
for i in syll:
    print(i)
    anova_results = ow_anova(temp, i, 'age')
    #results[i] = anova_results
    #print(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))
    cp = anova_results['PR(>F)']['age']*n
    if cp<a:
        s.extend([i])
        p.extend([cp])
print('number of syllables changing over time for females:', str(len(s)))

0
+-----------+------------+------+----------+---------------+
|           |     sum_sq |   df |        F |        PR(>F) |
|-----------+------------+------+----------+---------------|
| Intercept | 0.00228621 |    1 | 119.004  |   4.01756e-24 |
| age       | 0.00115124 |    1 |  59.9257 |   1.01223e-13 |
| Residual  | 0.00689681 |  359 | nan      | nan           |
+-----------+------------+------+----------+---------------+
2
+-----------+-------------+------+-----------+---------------+
|           |      sum_sq |   df |         F |        PR(>F) |
|-----------+-------------+------+-----------+---------------|
| Intercept | 0.107939    |    1 | 399.227   |   3.00484e-60 |
| age       | 0.000816066 |    1 |   3.01832 |   0.0831864   |
| Residual  | 0.0970632   |  359 | nan       | nan           |
+-----------+-------------+------+-----------+---------------+
4
+-----------+-----------+------+-----------+---------------+
|           |    sum_sq |   df |         F |        PR(>F) |
|---

+-----------+-------------+------+-----------+---------------+
|           |      sum_sq |   df |         F |        PR(>F) |
|-----------+-------------+------+-----------+---------------|
| Intercept | 0.0037367   |    1 | 153.929   |   1.16636e-29 |
| age       | 5.78004e-05 |    1 |   2.38102 |   0.123698    |
| Residual  | 0.00871489  |  359 | nan       | nan           |
+-----------+-------------+------+-----------+---------------+
36
+-----------+------------+------+------------+---------------+
|           |     sum_sq |   df |          F |        PR(>F) |
|-----------+------------+------+------------+---------------|
| Intercept | 0.151128   |    1 | 1214.41    |   3.0625e-117 |
| age       | 0.00058209 |    1 |    4.67748 |   0.0312202   |
| Residual  | 0.0446758  |  359 |  nan       | nan           |
+-----------+------------+------+------------+---------------+
39
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |     

+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.000848039 |    1 | 120.516  |   2.26826e-24 |
| age       | 0.000101384 |    1 |  14.4079 |   0.000172729 |
| Residual  | 0.00252618  |  359 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
77
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.00161051  |    1 | 139.152  |   2.29626e-27 |
| age       | 0.000121311 |    1 |  10.4815 |   0.00131807  |
| Residual  | 0.00415499  |  359 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
79
+-----------+-------------+------+-----------+---------------+
|           |      sum_sq |   df |         F |        PR(>F) |


In [10]:
#males
data=f_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

# Initialize variables
results = []
dependent_vars = data.columns
n_tests = len(dependent_vars)  # Number of ANOVA tests

# Perform ANOVA for each dependent variable
for i, k in enumerate(dependent_vars):
    print(k)
    anova_results = ow_anova(temp, k, 'age')
    anova_results['Dependent Variable'] = k  # Add dependent variable to the results
    results.append(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))

# Combine results into a DataFrame
results_df = pd.concat(results).reset_index()
results_df.rename(columns={'index': 'Parameter'}, inplace=True)

# Move the 'Dependent Variable' column to the first position
cols = ['Dependent Variable'] + [col for col in results_df.columns if col != 'Dependent Variable']
results_df = results_df[cols]

# Apply Bonferroni correction to the p-values
results_df['Bonferroni_corrected_p'] = results_df['PR(>F)'] * n_tests
results_df['Bonferroni_corrected_p'] = results_df['Bonferroni_corrected_p'].apply(lambda p: min(p, 1))  # p-values should not exceed 1

# Save DataFrame to CSV file
results_df.to_csv('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/stats/anova_results_syllable_over_time_ontogeny_females.csv', index=False)

print("ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'")

0
+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.00228621 |    1 | 119.004  |   4.01756e-24 |                    0 |
| age       | 0.00115124 |    1 |  59.9257 |   1.01223e-13 |                    0 |
| Residual  | 0.00689681 |  359 | nan      | nan           |                    0 |
+-----------+------------+------+----------+---------------+----------------------+
2
+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.107939    |    1 | 399.227   |   3.00484e-60 |                    2 |
| age       | 0.000816066 |    1 |   3.01832 |   0.0831864   |  

+-----------+-------------+------+-------------+---------------+----------------------+
|           |      sum_sq |   df |           F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-------------+---------------+----------------------|
| Intercept | 0.195604    |    1 | 853.223     |   6.85817e-97 |                   23 |
| age       | 8.94151e-06 |    1 |   0.0390028 |   0.843555    |                   23 |
| Residual  | 0.0823019   |  359 | nan         | nan           |                   23 |
+-----------+-------------+------+-------------+---------------+----------------------+
24
+-----------+-------------+------+--------------+---------------+----------------------+
|           |      sum_sq |   df |            F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+--------------+---------------+----------------------|
| Intercept | 0.00369965  |    1 | 151.209      |   3.04827e-29 |                   24 |
| age       | 6.14302e-08

+-----------+-------------+------+----------+---------------+----------------------+
|           |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+---------------+----------------------|
| Intercept | 0.00142135  |    1 | 121.71   |   1.44631e-24 |                   54 |
| age       | 0.000470648 |    1 |  40.3015 |   6.57475e-10 |                   54 |
| Residual  | 0.00419246  |  359 | nan      | nan           |                   54 |
+-----------+-------------+------+----------+---------------+----------------------+
55
+-----------+-------------+------+----------+---------------+----------------------+
|           |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+---------------+----------------------|
| Intercept | 0.00386592  |    1 | 346.57   |   1.26973e-54 |                   55 |
| age       | 0.000124674 |    1 |  11.1768 |   0.000915545 | 

+-----------+-------------+------+--------------+---------------+----------------------+
|           |      sum_sq |   df |            F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+--------------+---------------+----------------------|
| Intercept | 0.00122395  |    1 |  90.4539     |   2.81868e-19 |                   73 |
| age       | 8.21102e-08 |    1 |   0.00606819 |   0.937952    |                   73 |
| Residual  | 0.00485772  |  359 | nan          | nan           |                   73 |
+-----------+-------------+------+--------------+---------------+----------------------+
74
+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.00113422 |    1 |  16.5644 |   5.78682e-05 |                   74 |
| age       | 0.00406536 |    1 |  59.

In [11]:
## sex-specific change in syllable over age 2-way anova

In [12]:
data = pd.concat([m_data,f_data])
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
ss=[]
ps=[]
ints=[]
intp=[]
results={}
a = 0.05
from scipy import stats
for i in syll:
    print(i)
    anova_results = tw_anova(temp, i, 'age','sex')
    #print(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))
    #results[i] = anova_results
    cp = anova_results['PR(>F)']['C(sex)']*n
    if cp<a:
        ss.extend([i])
        ps.extend([cp])
    cp = anova_results['PR(>F)']['age:C(sex)']*n
    if cp<a:
        ints.extend([i])
        intp.extend([cp])
print('number of syllables different between males and females:', str(len(ss)))
print('number of syllables with sex/age interactions: ', str(len(ints)))

0
+------------+-------------+------+------------+---------------+
|            |      sum_sq |   df |          F |        PR(>F) |
|------------+-------------+------+------------+---------------|
| Intercept  | 0.00135463  |    1 |  93.2285   |   1.1559e-20  |
| C(sex)     | 8.30939e-07 |    1 |   0.057187 |   0.811076    |
| age        | 0.00153591  |    1 | 105.704    |   5.013e-23   |
| age:C(sex) | 0.000257331 |    1 |  17.7101   |   2.94673e-05 |
| Residual   | 0.00915404  |  630 | nan        | nan           |
+------------+-------------+------+------------+---------------+
2
+------------+-------------+------+-------------+---------------+
|            |      sum_sq |   df |           F |        PR(>F) |
|------------+-------------+------+-------------+---------------|
| Intercept  | 0.087706    |    1 | 333.505     |   4.06819e-60 |
| C(sex)     | 3.43283e-06 |    1 |   0.0130535 |   0.909075    |
| age        | 0.00137678  |    1 |   5.23526   |   0.0224639   |
| age:C(sex) | 

+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 5.72457e-05 |    1 |   3.60635 |   0.0580157   |
| C(sex)     | 0.000497513 |    1 |  31.3422  |   3.23139e-08 |
| age        | 0.00177844  |    1 | 112.038   |   3.28773e-24 |
| age:C(sex) | 6.12111e-05 |    1 |   3.85617 |   0.0500027   |
| Residual   | 0.0100003   |  630 | nan       | nan           |
+------------+-------------+------+-----------+---------------+
27
+------------+-------------+------+------------+---------------+
|            |      sum_sq |   df |          F |        PR(>F) |
|------------+-------------+------+------------+---------------|
| Intercept  | 0.0020017   |    1 | 118.733    |   1.89501e-25 |
| C(sex)     | 0.00032679  |    1 |  19.384    |   1.25571e-05 |
| age        | 0.000136457 |    1 |   8.09411  |   0.0045852   |
| age:C(sex) | 3.50064e-06 |   

+------------+-------------+------+-------------+---------------+
|            |      sum_sq |   df |           F |        PR(>F) |
|------------+-------------+------+-------------+---------------|
| Intercept  | 0.00139596  |    1 |   9.85403   |   0.00177382  |
| C(sex)     | 0.00223607  |    1 |  15.7843    |   7.91925e-05 |
| age        | 1.00719e-05 |    1 |   0.0710971 |   0.789833    |
| age:C(sex) | 0.00238177  |    1 |  16.8128    |   4.66592e-05 |
| Residual   | 0.0892482   |  630 | nan         | nan           |
+------------+-------------+------+-------------+---------------+
60
+------------+-------------+------+------------+---------------+
|            |      sum_sq |   df |          F |        PR(>F) |
|------------+-------------+------+------------+---------------|
| Intercept  | 0.00232831  |    1 | 354.187    |   4.9623e-63  |
| C(sex)     | 3.4296e-06  |    1 |   0.521716 |   0.470378    |
| age        | 0.000163811 |    1 |  24.9191   |   7.74758e-07 |
| age:C(sex) 

+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 0.00409083  |    1 |  74.2698  |   5.48759e-17 |
| C(sex)     | 0.000131042 |    1 |   2.3791  |   0.12347     |
| age        | 0.000271924 |    1 |   4.93684 |   0.0266446   |
| age:C(sex) | 6.81872e-05 |    1 |   1.23795 |   0.26629     |
| Residual   | 0.0347008   |  630 | nan       | nan           |
+------------+-------------+------+-----------+---------------+
83
+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 0.00191904  |    1 |  75.6351  |   2.95798e-17 |
| C(sex)     | 0.000118317 |    1 |   4.66321 |   0.0311922   |
| age        | 0.000421776 |    1 |  16.6234  |   5.14227e-05 |
| age:C(sex) | 0.000103147 |    1 |  

In [13]:
data = pd.concat([m_data,f_data])
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
results={}
a = 0.05

# Initialize variables
results = []
dependent_vars = data.columns
n_tests = len(dependent_vars)  # Number of ANOVA tests

# Perform ANOVA for each dependent variable
for i, k in enumerate(dependent_vars):
    print(k)
    anova_results = tw_anova(temp, k, 'age', 'sex')
    anova_results['Dependent Variable'] = k  # Add dependent variable to the results
    results.append(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))

# Combine results into a DataFrame
results_df = pd.concat(results).reset_index()
results_df.rename(columns={'index': 'Parameter'}, inplace=True)

# Move the 'Dependent Variable' column to the first position
cols = ['Dependent Variable'] + [col for col in results_df.columns if col != 'Dependent Variable']
results_df = results_df[cols]

# Apply Bonferroni correction to the p-values
results_df['Bonferroni_corrected_p'] = results_df['PR(>F)'] * n_tests
results_df['Bonferroni_corrected_p'] = results_df['Bonferroni_corrected_p'].apply(lambda p: min(p, 1))  # p-values should not exceed 1

# Save DataFrame to CSV file
results_df.to_csv('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/stats/anova_results_fvm_ontogeny.csv', index=False)

print("ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'")

0
+------------+-------------+------+------------+---------------+----------------------+
|            |      sum_sq |   df |          F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+------------+---------------+----------------------|
| Intercept  | 0.00135463  |    1 |  93.2285   |   1.1559e-20  |                    0 |
| C(sex)     | 8.30939e-07 |    1 |   0.057187 |   0.811076    |                    0 |
| age        | 0.00153591  |    1 | 105.704    |   5.013e-23   |                    0 |
| age:C(sex) | 0.000257331 |    1 |  17.7101   |   2.94673e-05 |                    0 |
| Residual   | 0.00915404  |  630 | nan        | nan           |                    0 |
+------------+-------------+------+------------+---------------+----------------------+
2
+------------+-------------+------+-------------+---------------+----------------------+
|            |      sum_sq |   df |           F |        PR(>F) |   Dependent Variable |
|------------+------------

+------------+-------------+------+------------+---------------+----------------------+
|            |      sum_sq |   df |          F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+------------+---------------+----------------------|
| Intercept  | 0.0217942   |    1 | 309.839    |   1.05219e-56 |                   14 |
| C(sex)     | 0.000478784 |    1 |   6.80665  |   0.00929745  |                   14 |
| age        | 1.78961e-05 |    1 |   0.254421 |   0.614155    |                   14 |
| age:C(sex) | 0.00289688  |    1 |  41.1836   |   2.72558e-10 |                   14 |
| Residual   | 0.0443145   |  630 | nan        | nan           |                   14 |
+------------+-------------+------+------------+---------------+----------------------+
16
+------------+-------------+------+------------+---------------+----------------------+
|            |      sum_sq |   df |          F |        PR(>F) |   Dependent Variable |
|------------+-------------+-

+------------+-------------+------+-----------+---------------+----------------------+
|            |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+-----------+---------------+----------------------|
| Intercept  | 0.000396883 |    1 |  14.9972  |   0.000118922 |                   39 |
| C(sex)     | 0.00042074  |    1 |  15.8986  |   7.46603e-05 |                   39 |
| age        | 8.24144e-05 |    1 |   3.11422 |   0.0780961   |                   39 |
| age:C(sex) | 0.000170644 |    1 |   6.44818 |   0.0113459   |                   39 |
| Residual   | 0.0166722   |  630 | nan       | nan           |                   39 |
+------------+-------------+------+-----------+---------------+----------------------+
41
+------------+-------------+------+----------+---------------+----------------------+
|            |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+-------

+------------+------------+------+----------+---------------+----------------------+
|            |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|------------+------------+------+----------+---------------+----------------------|
| Intercept  | 0.00901758 |    1 | 101.399  |   3.24066e-22 |                   61 |
| C(sex)     | 0.00148454 |    1 |  16.6931 |   4.9617e-05  |                   61 |
| age        | 0.00121171 |    1 |  13.6252 |   0.000242504 |                   61 |
| age:C(sex) | 0.002563   |    1 |  28.8198 |   1.11816e-07 |                   61 |
| Residual   | 0.0560269  |  630 | nan      | nan           |                   61 |
+------------+------------+------+----------+---------------+----------------------+
62
+------------+------------+------+------------+----------------+----------------------+
|            |     sum_sq |   df |          F |         PR(>F) |   Dependent Variable |
|------------+------------+------+------------+---------

+------------+-------------+------+----------+---------------+----------------------+
|            |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+----------+---------------+----------------------|
| Intercept  | 0.00106111  |    1 | 103.028  |   1.59717e-22 |                   76 |
| C(sex)     | 0.00030801  |    1 |  29.9063 |   6.54462e-08 |                   76 |
| age        | 0.000281504 |    1 |  27.3327 |   2.33333e-07 |                   76 |
| age:C(sex) | 4.57779e-05 |    1 |   4.4448 |   0.0354019   |                   76 |
| Residual   | 0.00648849  |  630 | nan      | nan           |                   76 |
+------------+-------------+------+----------+---------------+----------------------+
77
+------------+-------------+------+-------------+---------------+----------------------+
|            |      sum_sq |   df |           F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+----------